# Data Structures and Algorithms in Python - Dynamic Programming
### AJ Zerouali, 2023/10/25

## 0) Introduction

**References:**

- "Introduction to Algorithms", by Cormen, Leiserson, Rivest, and Stein (primary, abbreviated [CLRS22]). Dynamic Programming is covered in Ch.14.
- "Data structures and algorithms in Python", by Goodrich, Tamassia and Goldwasser (secondary, abbreviated [GTG13]). Section we'll follow is 13.3.
- Portilla does not cover dynamic programming, except for one problem in the recursion section.
- The DSA course by Elshad Karimov covers dynamic programming in section 47, with challenging problems in section 48.

The (tentative) contents of this notebook are as follows:
- The rod cutting problem [CLRS22, Sec.14.1].
- Matrix-chain multiplication [CLRS22, Sec.14.2].
- Dynamic programming in general [CLRS22, Sec.14.3].
- Longest common subsequence [CLRS22, Sec.14.4].

In some sense, dynamic programming is a mix of divide-and-conquer and recursion, where we solve subproblems to obtain the solution of the main problem. There are two issues that make dynamic programming delicate:
1) Of course, this is not applicable in every setting, because as in optimal control, there needs to be a "principle of optimality": the main problem should be able to be formulated in such a way that the subproblems have a well-defined optimal solution.
2) When implementing a dynamic programming algorithm, special care needs to be taken to ensure that similar subproblems are not solved repeatedly. This motivates the use of memoization.

## 1) Rod cutting

Here we summarize [CLRS22, Sec.14.1].

### 1.a - Problem formulation

A company cuts steel rods and sells the parts obtained at various prices. The company has a table of prices $p_i$ (in dollars) as a function of the length $i=1,2,\cdots,n,\cdots$ (in inches) of the rod. Given a rod of length $n$, what is the optimal revenue $r_n$ that can be achieved by cutting the rod into smaller pieces and selling them?

For the sake of concreteness, suppose we have the following table of prices:

        Length i (in)  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
        ---------------------------------------------------------
        Price ($)      | 1 | 5 | 8 | 9 | 10| 17| 17| 20| 24| 30 |

### 1.b - Preliminaries and naive solution

For simplicity, we will consider that cutting the rod corresponds to choosing the lengths $i_j$ starting from the left at which we perform cuts. The resulting ways of cutting will not be distinct, but we address this issue later. There are $(n-1)$ cutting positions to choose, and summing over the total number of cuts $j$ to make we have:

$$\sum_{j=0}^{n-1}C_j^{n-1}=2^{n-1}$$

ways of cutting a rod of length $n$. An important observation here is that the rod cutting problem exhibits **optimal substructure**, since the maximal revenue $r_n$ of cutting a rod of length $n$ depends on the maximal revenue $r_{n-k}$ of cutting a rod of length $(n-k)$, with $k=0,\cdots, n$. In fact continuing with the notation of the previous section, it is easy to see that:

$$r_n = \max_{k=0,\cdots,n-1}\{p_k+r_{n-k}\}.$$

Using this last formula, we can readily solve the rod-cutting problem with the following top-down 

implementation:

In [3]:
# Index is length minus one
P = [1,5,8,9,10,17,17,20,24,30]

In [2]:
def cut_rod(tab, n):
    if n==0:
        return 0
    q = -20
    for i in range(n):
        q = max(q, tab[i]+cut_rod(tab, n-(i+1)))
    return q
    

As shown in [CLRS22, Fig.14.2], the optimal revenue for a rod of length 4 is 10$:

In [5]:
cut_rod(P,4)

10

There is a major issue with this solution though. Suppose that $T(n)$ is the number of calls to the *cut_rod()* function. It is easily established that $T(j)=2^j$ for $j=0,1,2$, and from our max formula for $r_n$, it is easily seen that $T(n)=1+\sum_{j=0}^{n-1}T(j)$, and consequently: $T(n)=2^n$ with this implementation. Therefore, we expect the *cut_rod()* algorithm to run in exponential time $O(2^n)$.

This is highly inefficient, and we can easily see that several calls of the form *cut_rod(P,k)* are duplicated. In fact, as discussed in [CLRS22, Fig.14.3], this top-down implementation builds a recursion tree with $2^n$ nodes with $2^{n-1}$ leaves (each node corresponds to a distinct call to the function). To resolve this optimality issue, we next explore alternative implementations.

### 1.c - Bottom-up implementation

One aspect of dynamic programming that wasn't previously implemented is the *memoization*, or the use of a table for previously computed values, and which is the origin of the "programming" part of the name.

Instead of recursively calling *cut_rod()* for each new length, we instead compute and store $r_k$ starting from the base case and with $k$ increasing from $1$ to $n$. The computation is hence from the bottom upwards in $k$, with an array storing the previously computed values:

In [4]:
def bottomup_cut_rod(tab, n):
    # Initialization of cache
    ## Index i is r_i
    r= [None]*(n+1)
    r[0] = 0
    
    # Loop over length k
    for k in range(1,n+1):
        q = -100
        
        # Loop over position of first cut
        for i in range(1,k+1):
            q = max(q, tab[i-1]+r[k-i]) # Index is shifted by 1
        
        # Store current r_k
        r[k] = q
    
    # Output
    return r[n]

In [5]:
# Index is length minus one
P = [1,5,8,9,10,17,17,20,24,30]

In [6]:
bottomup_cut_rod(P,4)

10

For comparison with the results on p.364 of [CLRS22]:

In [8]:
for n in range(1,11):
    r_n = bottomup_cut_rod(P, n)
    print(f"Optimal revenue for n={n}: r_n = {r_n}")

Optimal revenue for n=1: r_n = 1
Optimal revenue for n=2: r_n = 5
Optimal revenue for n=3: r_n = 8
Optimal revenue for n=4: r_n = 10
Optimal revenue for n=5: r_n = 13
Optimal revenue for n=6: r_n = 17
Optimal revenue for n=7: r_n = 18
Optimal revenue for n=8: r_n = 22
Optimal revenue for n=9: r_n = 25
Optimal revenue for n=10: r_n = 30


This implementation is by far better than the naive one, and it is easily seen that it runs in $O(n^2)$ time. It's worth noting here that this only gives the optimal revenue **value**, but not the optimal solution, i.e. the list of cuts giving the optimal solution. See [CLRS22, p.372] for an extended implementation.

### 1.d - Top-down implementation with memoization

This implementation is the same as the naive top-down one, except now we add an array to store the previously computed optimal revenues.

In [12]:
def memoized_cut_rod(tab, n, R = None):
    if R is None:
        R = [None]*(n+1)
        R[0] = 0
        
    if R[n] is not None:
        return R[n]
    else:
        q = -20
        for i in range(1,n+1):
            q = max(q, tab[i-1]+memoized_cut_rod(tab, n-i, R))
        R[n]=q
        return q

In [14]:
memoized_cut_rod(P, 4)

10

In [15]:
for n in range(1,11):
    r_n = memoized_cut_rod(P, n)
    print(f"Optimal revenue for n={n}: r_n = {r_n}")

Optimal revenue for n=1: r_n = 1
Optimal revenue for n=2: r_n = 5
Optimal revenue for n=3: r_n = 8
Optimal revenue for n=4: r_n = 10
Optimal revenue for n=5: r_n = 13
Optimal revenue for n=6: r_n = 17
Optimal revenue for n=7: r_n = 18
Optimal revenue for n=8: r_n = 22
Optimal revenue for n=9: r_n = 25
Optimal revenue for n=10: r_n = 30


## 2) Matrix-chain multiplication

This part summarizes [CLRS22, Sec.14.2]. Before stating the problem, we introduce the following vocabulary: A product of matrices is said to be **fully parenthesized** if it is a single matrix or the product of two fully parenthesized matrix product. For example, if $\{A_1, A_2, A_3\}$ is a chain of matrices that can be contiguously multiplied, there are two possible full parenthetizations of the product $A_1 A_2 A_3$: $((A_1 A_2)A_3)$ and $(A_1 (A_2 A_3))$.

A second point that is assumed in this section is the algorithm used for matrix multiplication. We will assume that we work with an algorithm based on the following pseudocode:

        Algorithm Matrix_Mult(A,B):
            # Get shapes of matrices
            (p,q) = shape(A)
            (q,r) = shape(B)
            
            # Init. C = A.B
            C = zero_matrix(shape = (p,r))
            
            # Compute entries of C
            for i = 1, ..., p:
                for j = 1, ..., r:
                    for k = 1, ..., q:
                        c[i,j] = c[i,j]+a[i,k]*b[k,j]
            
            # Output
            return C

Thus, multiplying $A\in\mathbb{M}_{p\times q}$ and $B\in\mathbb{M}_{q\times r}$ has $O(pqr)$ time complexity.

### 2.a - Problem formulation and discussion

Suppose we have a chain of matrices $\{A_1,A_2,\cdots,A_n\}$ such that $A_i$ has shape $p_{i-1}\times p_i$ ($i=1,\cdots n$). Assuming that computing the product of two matrices $A\in\mathbb{M}_{p\times q}$ and $B\in\mathbb{M}_{q\times r}$ costs $p\cdot q\cdot r$ scalar multiplications, what is the parenthetization of the product $\prod_{i=1}^n A_i$ that minimizes the number of scalar multiplications?


#### Example:
To illustrate why this is an important question, suppose $A_1\in \mathbb{M}_{10\times 100}$, $A_2\in \mathbb{M}_{100\times 5}$, and $A_3\in \mathbb{M}_{5\times 50}$. Let's count the total number of scalar multiplications for each parenthesization:

1) $((A_1A_2)A_3)$: Computing $(A_1A_2)$ will involve $5000$ multiplications, and then multiplying by $A_3$ will involve $2500$ multiplications, so the total number of multiplications for this parenthetization is $7500$.

2) $(A_1(A_2A_3))$: The inner parentheses involve $25000$ multiplications, and the product with $A_1$ will add $50000$ multiplications for a total of $75000$.

Thus, choosing the correct parenthesization reduces the number of scalar multiplications by $90\%$ in this case.

To get an idea of the complexity of this problem, it could be useful to first find the number of possible parenthesizations. By induction, one can show that for $n$ matrices, there are $C_n$ parenthesizations, where the latter denotes the $n$-th Catalan number, where $C_1=1$ and

$$C_n = \sum_{k=1}^{n-1}C_k\cdot C_{n-k},\ \ \forall n\ge 2.$$
 
This grows as $\Omega(4^n/n^{3/2})$, meaning that checking all possible parenthesizations to find the minimum is essentially unfeasible.

A more clever approach is to notice the following in the case of $n=5$: The parenthesizations can be done in a first step in 4 different ways:
* $A_1(A_2A_3A_4A_5)$
* $(A_1A_2)(A_3A_4A_5)$
* $(A_1A_2A_3)(A_4A_5)$
* $(A_1A_2A_3A_4)A_5$

In each of the cases above, we first find the optimal parenthesization of the shorter chains (of length 3 and 4), and then we take the minimum over the 4 numbers obtained. To formalize this idea, suppose $1\le i\le j \le 5$, and let $m(i,j)$ denote the **minimal total number of multiplications** over all parenthesizations of $A_iA_{i+1}\cdots A_j$. We then have $m(i,j)=0$ if $i=j$, and from the above:

$$m(1,5)= \min_{k=1,\cdots,4} \{m(1,k)+m(k+1,5)+p_0p_{k}p_5\},$$

and more generally, for each of the sub-parenthesizations, this leads us to:

$$m(i,j)= \min_{k=i,\cdots,j-1} \{m(i,k)+m(k+1,j)+p_{i-1}p_{k}p_j\},\ \ \forall i<j.$$

In other words, this recursive expression gives us the optimal substructure for this problem.

Next, when it comes to the implementation of the function, we need not actually use the matrices ${A_1,\cdots, A_n}$, as we can instead work with the array of dimensions $[p_0,p_1,\cdots,p_n]$ as the input. The subproblems are however solved using indices $(i,j)$.

### 2.b - Bottom-up implementation of the solution

In the implementation below, we not only return the final minimal value, but also the table of all optimal values for subproblems, as well as the optimal solution. Here, for a given chain $(i,j)$, the optimal solution is the index $i\le k<j$.

In [4]:
def min_matrix_chain(dims):
    # Num of matrices in chain
    n = len(dims)-1
    if n<2:
        raise ValueError("dims must contain at least 3 elements")
    elif n==2:
        return dims[0]*dims[1]*dims[2], {(1,2):dims[0]*dims[1]*dims[2]}, {(1,2):1}
        
    elif n>2:
        # Init. dict of values
        mem = {}
        # Init. dict of soln's
        sol = {}
        # Start with chains of len = 0
        for i in range(1,n+1):
            mem[(i,i)]=0
        
        # Loop over lengths l=2,...,n
        for l in range(2,n+1):
            #print(f"starting length l = {l}")
            # Loop over starting index
            ## Last possible start index is n-l+1
            for i in range(1,(n-l+1)+1):
                # Last index in [i,j] chain
                j = i + l-1
                # Init m[(i,j)]
                mem[(i,j)]=float("inf")
                # Loop over k=i,...,j-1 for min.
                for k in range(i,j):
                    temp = mem[(i,k)]+mem[(k+1,j)]+dims[i-1]*dims[k]*dims[j]
                    if temp <= mem[(i,j)]:
                        mem[(i,j)] = temp
                        sol[(i,j)] = k
            
        # Output
        return mem[(1,n)], mem, sol
        

In [5]:
dims = [10,100,5,50]

In [6]:
m, mem, sol = min_matrix_chain(dims)

In [23]:
m

7500

In [7]:
mem

{(1, 1): 0, (2, 2): 0, (3, 3): 0, (1, 2): 5000, (2, 3): 25000, (1, 3): 7500}

In [8]:
sol

{(1, 2): 1, (2, 3): 2, (1, 3): 2}

## 3) Number Factor Problem

This is taken from Karimov's "Complete DSA in Python", lectures 447, 448, 470 and 471.

### 3.a - Problem formulation

Given an integer $N$, how many ways are there to express it as a sum of the numbers $\{1, 3, 4\}$, including permutations?

This is essentially a variation of the coin change problem, where instead of looking for the minimal number of factors, we count the number of partitions.

#### Example 1:

Say $N= 4$, then:

$$
4 = 1 +1 +1+1\\ 
= 1+3\\
= 3+1\\
= 4\\
$$

Thus, the partitions of $4$ using the given set are $\{(4),(1,3),(3,1),(1,1,1,1)\}$. If $F$ denotes the counting function for this problem, then $F(4)=4$.

#### Example 2:

Say $N=5$. Let's count the partitions (with permutations) as follows: For each element $i$ of the list $\{1,3,4\}$, in how many ways can we obtain partitions of 5 by adding $i$ to the partitions of $(N-i)$?

* For $i=1$: In this case we have $F(4)=4$ partitions:
$$\{(4,1),(1,3,1),(3,1,1),(1,1,1,1,1)\}.$$

* For $i=3$: In this case we have $F(2)=1$ partitions:
$$\{(1,1,3)\}.$$

* For $i=4$: In this case we have $F(1)=1$ partitions:
$$\{(1,4)\}.$$

### 3.b - Comments on the solution

Looking at example 2 above, it is easily seen that a solution to the number factor problem can be solved with a recursion, since the problem has an overlapping sub-problem structure. However, one needs to be careful with the fact that the same subproblem can be solved many times. Again, considering the $\{1,3,4\}$ sequence, computing $F(8)$ leads to the following recursive tree when following the approach in example 2:

                         F(8)
               /          |          \
           F(4)         F(5)           F(7)
       /   |   \     /   |   \      /   |   \
    F(0) F(1) F(3) F(1) F(2) F(4) F(3) F(4) F(6)

Here is a first simplified implementation, where we keep the list of numbers *[1,3,4]* as our "factors", and where we only use recursion.

In [1]:
# Easy case with nums = [1,3,4]
def SmallNumFactors(N):
    if N in [0,1,2]:
        return 1
    elif N==3:
        return 2
    elif N>=4:
        return SmallNumFactors(N-4) + SmallNumFactors(N-3) + SmallNumFactors(N-1)

In [2]:
SmallNumFactors(4)

4

In [3]:
SmallNumFactors(5)

6

Now we turn to the dynamic programming versions for larger numbers. First, using memoization, we have the following implementation:

In [8]:
# Easy case with nums = [1,3,4]
def SmallNumFactorsDPmem(N, mem=None):
    
    if mem is None:
        if N in [0,1,2]:
            return 1
        elif N==3:
            return 2
        elif N>=4:
            mem = [None]*(N+1)
            mem[0] = 1
            mem[1] = 1
            mem[2] = 1
            mem[3] = 2

    if mem[N] is None:
        mem[N] = SmallNumFactorsDPmem(N-4,mem)+SmallNumFactorsDPmem(N-3,mem)+SmallNumFactorsDPmem(N-1,mem)
    
    return mem[N]


In [14]:
SmallNumFactorsDPmem(8)

25

In [15]:
SmallNumFactorsDPmem(5)

6

Next, the bottom-up approach can be done as follows

In [17]:
# Easy case with nums = [1,3,4]
def SmallNumFactorsDPBU(N):
    nums = [1,3,4]
    if N in [0,1,2]:
        return 1
    elif N == 3:
        return 2
    elif N>=4:
        M = [0]*(N+1)
        M[0] = 1
        M[1] = 1
        M[2] = 1
        M[3] = 2
        
        for n in range(4,N+1):
            for k in nums:
                M[n] += M[n-k]
        
        return M[N]


In [18]:
SmallNumFactorsDPBU(5)

6

Here we solved this problem with a fixed set of factors *[1,3,4]*. If the list of factors was assumed to also be variable, the solution would have been much more delicate to implement.

## 3) Dynamic Programming in general

This part is based on [CLRS22, Sec.14.3]. To be able to solve a problem using dynamic programming, there are essentially two ingredients that have to be present: An optimal substructure, and overlapping subproblems.

In view of the other algorithms studied up to this point, it is useful to view dynamic programming as a form of divide-and-conquer procedure used when we have overlapping problems and optimal substructure. Very often also, dynamic programming is more easily implemented using recursions, which leads us to the discussion of memoization and/or bottom-up approaches.

### 3.a - Optimal substructure

In each of the previously treated problems, we constructed a final solution based on optimal solutions to subproblems. The pattern we used is the following:
1) We make an initial initial choice, which gives rise to a "subproblem", by which we mean solving the same problem but "lower dimension" or less parameters. (E.g., a shorter input array, or a lower value of an amount).
2) For a given problem, we assume that we are given the choice leading to an optimal solution. At this step we do not determine it.
3) Given an optimal choice, we determine the ensuing subproblems, and characterize their "structure".
4) We show that the solutions to the subproblems used within an optimal solution must themselves be optimal, essentially checking a principle of optimality.

There are two ways in which optimal substructure varies accross problems: (1) How many subproblems an optimal solution to the problem uses, and (2) how many choices need to be made to determine the subproblems solved. Both of these will determine the running time of the function that solves the problem.

To summarize, when we are looking for optimal substructure, we are essentially doing the divide-and-conquer part of this class of problems.

### 3.b - Overlapping subproblems

For dynamic programming to be applicable, the top-down recursive approach should inevitably lead to the same subproblems being solved, exactly as in the case of the number factor problem. In the case of more general divide-and-conquer algorithms, the subproblems generated by the "divide" part will essentially always be new, in which case dynamic programming is not applicable.

Given that we have overlapping subproblems, the "programming" part of dynamic programming essentially means that we memorize the subproblems once we solve them, and by re-using the solutions, we greatly improve the execution time of the implemented function. 

For a top-down implementation, keeping an array of results is called **memoization**. This array is checked at each recursive call, and typically for these implementations, some care needs to be taken in the initialization of the array and how the existence of a given entry is checked.

For bottom-up implementations, we always start by solving the subproblems and storing the optimal value. Of course, this means that we keep an array which is filled with solutions to base cases, and then build the solutions of the larger problems piece-by-piece. Here, when looking for a min or a max, it's useful to keep in mind that we often use an additional loop.

### 3.c - Top-down vs bottom-up

Both approaches have pros and cons:

1) Top-down approaches are slightly easier to implement once we understand how the optimal substructure works. In this case, the solution algorithm is recursive. The issue is that we need to be careful with the memoization, and to keep in mind that quite often, these algorithms are less efficient in terms of execution time.

2) Bottom-up approaches could be quite more efficient than bottom-down recursions, but they could be harder to implement. When building a bottom-up solution, we need to know more than the recursive properties of the solutions, which could be non-trivial in some cases.
